<a href="https://colab.research.google.com/github/nandhukumar86/GL-hackathons-1/blob/main/GL_Hackathon_RoughWork_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [100]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(color_codes=True)
%matplotlib inline

import os


In [101]:
os.chdir('D:\AIMLData\GL_Hackathon')

In [102]:
# fd_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/GL_Hackathon/frequency_domain_features_train.csv')
# fd_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/GL_Hackathon/frequency_domain_features_test.csv')
# td_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/GL_Hackathon/time_domain_features_train.csv')
# td_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/GL_Hackathon/time_domain_features_test.csv')
# hr_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/GL_Hackathon/heart_rate_non_linear_features_train.csv')
# hr_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/GL_Hackathon/heart_rate_non_linear_features_test.csv')

fd_train = pd.read_csv('frequency_domain_features_train.csv')
fd_test = pd.read_csv('frequency_domain_features_test.csv')
td_train = pd.read_csv('time_domain_features_train.csv')
td_test = pd.read_csv('time_domain_features_test.csv')
hr_train = pd.read_csv('heart_rate_non_linear_features_train.csv')
hr_test = pd.read_csv('heart_rate_non_linear_features_test.csv')

In [103]:
df3 = pd.merge(fd_train, td_train, how='outer', on='uuid')
df_train = pd.merge(df3, hr_train, how='outer', on='uuid')

In [104]:
df2 = pd.merge(fd_test, td_test, how='outer', on='uuid')
df_test = pd.merge(df2, hr_test, how='outer', on='uuid')

In [105]:
y_train = df_train.pop('HR')
X_train = df_train.copy(deep = True)
X_test = df_test.copy(deep = True)

In [106]:
X_train.drop(columns= ['datasetId'], inplace=True)
X_test.drop(columns= ['datasetId'], inplace=True)
X_train.drop(columns= ['uuid'], inplace=True)
y_id = X_test.pop('uuid')
X_train.drop(columns= ['SKEW_REL_RR','KURT_REL_RR'], inplace=True)
X_test.drop(columns= ['SKEW_REL_RR','KURT_REL_RR'], inplace=True)
X_train.drop(columns= ['SDSD_REL_RR'], inplace=True) 
X_test.drop(columns= ['SDSD_REL_RR'], inplace=True) 

In [107]:
cols = ['VLF', 'VLF_PCT', 'LF', 'LF_PCT', 'LF_NU', 'HF', 'HF_PCT', 'HF_NU',
       'TP', 'LF_HF', 'HF_LF', 'MEAN_RR', 'MEDIAN_RR', 'SDRR', 'RMSSD', 'SDSD',
       'SDRR_RMSSD', 'pNN25', 'pNN50', 'KURT', 'SKEW', 'MEAN_REL_RR',
       'MEDIAN_REL_RR', 'SDRR_REL_RR', 'RMSSD_REL_RR', 'SDRR_RMSSD_REL_RR',
       'SD1', 'SD2', 'sampen', 'higuci']

In [108]:
X_test['condition'].value_counts()

no stress        22158
interruption     11782
time pressure     7093
Name: condition, dtype: int64

# Feature Selection using RFECV

In [9]:
# from sklearn.feature_selection import RFECV
# from sklearn.linear_model import LassoCV, Lasso

# fs = RFECV(Lasso(), n_jobs= 8)

# sample = fs.fit_transform(X_train[cols], y_train)

In [10]:
# sample_columns = pd.DataFrame(fs.ranking_,cols)
# cols_rfecv = sample_columns[sample_columns[0]==1].index
# cols_rfecv

Index(['VLF_PCT', 'LF', 'HF', 'LF_HF', 'MEAN_RR', 'MEDIAN_RR', 'SD2'], dtype='object')

RFECV with Lasso: ['VLF_PCT', 'LF', 'HF', 'LF_HF', 'MEAN_RR', 'MEDIAN_RR', 'SD2']

In [109]:
cols_rfecv = ['VLF_PCT', 'LF', 'HF', 'LF_HF', 'MEAN_RR', 'MEDIAN_RR', 'SD2']

X_train = X_train[cols_rfecv]
X_test = X_test[cols_rfecv]

In [110]:
X_train['condition'] = hr_train['condition']
X_test['condition'] = hr_test['condition']

In [111]:
corr_matrix = X_train.corr()            
upper = corr_matrix.abs().where(np.triu(np.ones(corr_matrix.shape), k = 1).astype(np.bool))
high_correlation_features = [column for column in upper.columns if any(upper[column] > 0.6)]


In [112]:
high_correlation_features

['MEDIAN_RR']

In [113]:
X_train.drop(columns= ['MEDIAN_RR'], inplace=True)
X_test.drop(columns= ['MEDIAN_RR'], inplace=True)

In [114]:
categorical_columns = X_train.describe(include = ['O']).columns
numerical_columns = X_train.describe().columns

In [115]:
final_cols = ['VLF_PCT', 'LF', 'HF', 'LF_HF', 'MEAN_RR', 'SD2']

# Model

In [122]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import PowerTransformer, QuantileTransformer

pl_n = Pipeline([
                ('power', PowerTransformer())
                
])

pl_c = Pipeline([
                 ('std', OneHotEncoder(handle_unknown='ignore'))
])

from sklearn.compose import ColumnTransformer

ct = ColumnTransformer([
                        ('num', pl_n, numerical_columns),
                        ('cat', pl_c, categorical_columns)
])

In [123]:
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_absolute_error

m_model = ExtraTreesRegressor(n_jobs=-1, n_estimators = 10)

model = Pipeline([
               ('ct', ct),
               ('lr', m_model)
])

model.fit(X_train, y_train)

y_test_predict = model.predict(X_test)
y_train_predict = model.predict(X_train)

In [124]:
print('Training Score: ', '%0.8f'%mean_absolute_error(y_train, y_train_predict))

Training Score:  0.00000000


In [119]:
y_test_predict

array([64.70973409, 73.77877933, 69.19855018, ..., 60.94073368,
       77.40870147, 76.59353408])

In [120]:
df = pd.concat([pd.DataFrame(y_id),pd.DataFrame(np.round(y_test_predict,6))], axis=1, ignore_index=True)
df.columns = ['uuid','HR']
# df.to_csv('RF_ET750_Submission.csv', index=False)

In [69]:
def model_function(model, Xtrn, ytrn, Xtst, yid):
  model.fit(Xtrn, ytrn)
  ytstpredict = model.predict(Xtst)
  ytrnpredict = model.predict(Xtrn)
  print('Training Score: ', '%0.8f'%mean_absolute_error(ytrn, ytrnpredict))
  df1 = pd.concat([pd.DataFrame(yid),pd.DataFrame(np.round(ytstpredict,6))], axis=1, ignore_index=True)
  df1.columns = ['uuid','HR']
  return df1